In [ ]:
import sys
!{sys.executable} -m pip install kfp >/dev/null

In [ ]:
import os, json
import kfp
from kubernetes.client.models import V1EnvVar

In [ ]:
components_url = "https://raw.githubusercontent.com/oneconvergence/dkube/master/components/"
dkube_training_op = kfp.components.load_component_from_url(components_url + "training/component.yaml")
dkube_storage_op  = kfp.components.load_component_from_url(components_url + "storage/component.yaml")


In [ ]:
@kfp.dsl.pipeline(
    name='dkube-titanic-pl',
    description='example titanic pipeline to submit to leaderboard'
)
def titanic_pipeline(token, project_id):

    train       = dkube_training_op(token, '{"image":"ocdr/dkube-datascience-tf-cpu-multiuser:v2.0.0"}',
                                    framework="tensorflow", version="2.0.0",
                                    program="titanic", run_script="python train.py",
                                    datasets='["titanic-train"]', outputs='["titanic"]',
                                    input_dataset_mounts='["/titanic"]', ngpus=0,
                                    output_mounts='["/model"]',envs='[{"EPOCHS": "1"}]')

    input_volumes = json.dumps(["titanic-test-dataset@dataset://ocdkube:titanic-test",
                                "titanic-model@model://titanic",
                                "titanic-code@program://titanic"
                                ])
  

    storage = dkube_storage_op("export", token, namespace="kubeflow", input_volumes=input_volumes).after(train)
     


    predict_op = kfp.dsl.ContainerOp(
        name="predict",
        image="ocdr/dkube-datascience-tf-cpu-multiuser:v2.0.0",
        command=["python", "/code/predict.py"],
        pvolumes={"/dataset/": kfp.dsl.PipelineVolume(pvc="titanic-test-dataset"),
                 "/model/": kfp.dsl.PipelineVolume(pvc="titanic-model"),
                 "/code/": kfp.dsl.PipelineVolume(pvc="titanic-code")
                 },
        file_outputs={"output": "/tmp/prediction.csv"},
    ).after(storage)

    predictions = kfp.dsl.InputArgumentPath(predict_op.outputs["output"])
    
    submit_op = kfp.dsl.ContainerOp(
        name="submit",
        image="ocdr/d3project_eval",
        command=[
            "python",
            "submit.py",
            kfp.dsl.RUN_ID_PLACEHOLDER,
            "-t",
            token,
            predictions,
        ],
        file_outputs={
            "mlpipeline-ui-metadata": "/metadata.json",
            "results": "/results",
        },
    )
    env_var = V1EnvVar(name="DKUBE_PROJECT_ID", value=project_id)
    submit_op.add_env_variable(env_var)

In [ ]:
token  = os.getenv("DKUBE_USER_ACCESS_TOKEN")
client = kfp.Client(existing_token=token)
client.create_run_from_pipeline_func(titanic_pipeline, arguments={"token":token,"project_id":"pdd6vp"})